# Example of geometry optimization
## Batch of two molecules

In [2]:
import sys
### path to PYSEQM ###
sys.path.insert(1, "/home/maxim/Projects/git2/PYSEQM_dev/")

import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import Geometry_Optimization_SD
from seqm.seqm_functions.read_xyz import read_xyz
from seqm.seqm_functions.save_xyz import save_xyz


torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

/home/maxim/anaconda3/envs/hipnn_1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Decorating your function! <function KSA_XL_BOMD.one_step at 0x7f997d56fca0>


In [3]:
%%time

species = torch.as_tensor([[8,6,1,1],
                           [8,6,1,1],
                           [8,1,1,0]], # zero-padding for batching
                          dtype=torch.int64, device=device)

coordinates = torch.tensor([
                              [
                               [0.00,    0.00,    0.00],
                               [1.22,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [1.82,   -0.94,    0.00]
                              ],
                              [
                               [0.00,    0.00,    0.00],
                               [1.22,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [1.82,   -0.94,    0.00]
                              ],
                              [
                               [ 0.00,    0.00,    0.00],
                               [ 0.96,    0.00,    0.00],
                               [-0.45,   -0.57,    0.67],
                               [0.0,0.0,0.0]            # zero-padding for batching
                              ]
                            ], device=device)

const = Constants().to(device)
#may need to add scaling factor for length and energy on const, check constants.py

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

opt =  Geometry_Optimization_SD(seqm_parameters, alpha=0.008, force_tol=1.0e-2, max_evl=40).to(device)
max_force, dE =  opt.run(molecule)

Step,  Max_Force,      Etot(eV),     dE(eV)
1      1.796740e+00 ||-1.362046e+00 -1.362046e+00 ||-1.362046e+00 -1.362046e+00 ||-2.392370e+00 -2.392370e+00 
2      1.401060e+00 ||-1.363449e+00 -1.402605e-03 ||-1.363449e+00 -1.402605e-03 ||-2.449958e+00 -5.758791e-02 
3      1.153534e+00 ||-1.364404e+00 -9.549086e-04 ||-1.364404e+00 -9.549086e-04 ||-2.485704e+00 -3.574574e-02 
4      9.614630e-01 ||-1.364852e+00 -4.476405e-04 ||-1.364852e+00 -4.476405e-04 ||-2.510561e+00 -2.485719e-02 
5      8.052293e-01 ||-1.365099e+00 -2.469197e-04 ||-1.365099e+00 -2.469197e-04 ||-2.527962e+00 -1.740040e-02 
6      6.744733e-01 ||-1.365228e+00 -1.298780e-04 ||-1.365228e+00 -1.298780e-04 ||-2.540177e+00 -1.221528e-02 
7      5.646864e-01 ||-1.365311e+00 -8.232415e-05 ||-1.365311e+00 -8.232415e-05 ||-2.548744e+00 -8.566691e-03 
8      4.726775e-01 ||-1.365357e+00 -4.610113e-05 ||-1.365357e+00 -4.610113e-05 ||-2.554749e+00 -6.004965e-03 
9      3.954160e-01 ||-1.365386e+00 -2.896401e-05 ||-1.365386e+00 -2

### Final forces and optimized geometry

In [4]:
molecule.force

tensor([[[ 1.1220e-01,  2.1649e-14, -0.0000e+00],
         [-1.1985e-01, -1.1922e-13, -0.0000e+00],
         [ 3.8227e-03,  5.5900e-03, -0.0000e+00],
         [ 3.8227e-03, -5.5900e-03, -0.0000e+00]],

        [[ 1.1220e-01,  2.1649e-14, -0.0000e+00],
         [-1.1985e-01, -1.1922e-13, -0.0000e+00],
         [ 3.8227e-03,  5.5900e-03, -0.0000e+00],
         [ 3.8227e-03, -5.5900e-03, -0.0000e+00]],

        [[-8.8427e-04,  9.2631e-04, -1.0888e-03],
         [-1.2419e-03, -1.1381e-03,  1.3378e-03],
         [ 2.1261e-03,  2.1182e-04, -2.4899e-04],
         [-0.0000e+00, -0.0000e+00, -0.0000e+00]]], device='cuda:0')

In [5]:
molecule.coordinates

tensor([[[-1.1328e-03, -1.0836e-16,  0.0000e+00],
         [ 1.2253e+00,  5.3681e-16,  0.0000e+00],
         [ 1.8179e+00,  9.3940e-01,  0.0000e+00],
         [ 1.8179e+00, -9.3940e-01,  0.0000e+00]],

        [[-1.1328e-03, -1.0836e-16,  0.0000e+00],
         [ 1.2253e+00,  5.3681e-16,  0.0000e+00],
         [ 1.8179e+00,  9.3940e-01,  0.0000e+00],
         [ 1.8179e+00, -9.3940e-01,  0.0000e+00]],

        [[-3.8495e-02,  2.8406e-02, -3.3390e-02],
         [ 9.1661e-01, -4.1732e-02,  4.9053e-02],
         [-3.6811e-01, -5.5667e-01,  6.5434e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00]]], device='cuda:0',
       requires_grad=True)

### Save optimized geometries to .xyz files without final forces.

In [6]:
save_xyz(molecule, 'XYZ', Forces=False)

## Reading starting geometries from .xyz
### Only molecules of the same length are currently supported for batched xyz_reader.

In [7]:
%%time
torch.manual_seed(0)
files = ['coronene.xyz', 'coronene.xyz']

species, coordinates = read_xyz(files)
species = torch.as_tensor(species,dtype=torch.int64, device=device)[:]
coordinates = torch.tensor(coordinates, device=device)[:]
const = Constants().to(device)
#may need to add scaling factor for length and energy on const, check constants.py

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

opt =  Geometry_Optimization_SD(seqm_parameters, alpha=0.012, force_tol=1.0e-2, max_evl=40).to(device)
max_force, dE =  opt.run(molecule)

Step,  Max_Force,      Etot(eV),     dE(eV)
1      6.209117e-01 ||4.242853e+00 4.242853e+00 ||4.242853e+00 4.242853e+00 
2      3.736704e-01 ||4.186088e+00 -5.676525e-02 ||4.186088e+00 -5.676525e-02 
3      1.331077e-01 ||4.178330e+00 -7.758023e-03 ||4.178330e+00 -7.757888e-03 
4      1.012998e-01 ||4.176495e+00 -1.834690e-03 ||4.176493e+00 -1.837155e-03 
5      7.110089e-02 ||4.175770e+00 -7.258296e-04 ||4.175768e+00 -7.248595e-04 
6      6.372146e-02 ||4.175331e+00 -4.382871e-04 ||4.175330e+00 -4.381247e-04 
7      5.409485e-02 ||4.174989e+00 -3.418729e-04 ||4.174988e+00 -3.416071e-04 
8      5.027508e-02 ||4.174702e+00 -2.873046e-04 ||4.174701e+00 -2.870889e-04 
9      4.573716e-02 ||4.174461e+00 -2.415052e-04 ||4.174460e+00 -2.415047e-04 
10      4.420287e-02 ||4.174251e+00 -2.098692e-04 ||4.174250e+00 -2.100555e-04 
11      4.173205e-02 ||4.174071e+00 -1.793280e-04 ||4.174071e+00 -1.792012e-04 
12      4.157369e-02 ||4.173914e+00 -1.572261e-04 ||4.173914e+00 -1.570691e-04 
13     

### Alternatively, use zero-padding in .xyz files for molecules of different lengths.

In [8]:
%%time
torch.manual_seed(0)
files = ['coronene.xyz', 'benzene_zero_pad.xyz']

species, coordinates = read_xyz(files)
species = torch.as_tensor(species,dtype=torch.int64, device=device)[:]
coordinates = torch.tensor(coordinates, device=device)[:]
const = Constants().to(device)
#may need to add scaling factor for length and energy on const, check constants.py

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

opt =  Geometry_Optimization_SD(seqm_parameters, alpha=0.012, force_tol=1.0e-2, max_evl=40).to(device)
max_force, dE =  opt.run(molecule)

Step,  Max_Force,      Etot(eV),     dE(eV)
1      6.209117e-01 ||4.242853e+00 4.242853e+00 ||9.740017e-01 9.740017e-01 
2      3.736704e-01 ||4.186088e+00 -5.676525e-02 ||9.610107e-01 -1.299108e-02 
3      1.613409e-01 ||4.178331e+00 -7.757548e-03 ||9.573871e-01 -3.623593e-03 
4      9.913528e-02 ||4.176486e+00 -1.845041e-03 ||9.560962e-01 -1.290899e-03 
5      6.973712e-02 ||4.175764e+00 -7.215104e-04 ||9.555285e-01 -5.676752e-04 
6      6.273803e-02 ||4.175326e+00 -4.375414e-04 ||9.552564e-01 -2.720852e-04 
7      5.321673e-02 ||4.174986e+00 -3.408211e-04 ||9.551178e-01 -1.386395e-04 
8      4.951309e-02 ||4.174699e+00 -2.869109e-04 ||9.550473e-01 -7.043321e-05 
9      4.506003e-02 ||4.174458e+00 -2.410167e-04 ||9.550108e-01 -3.650346e-05 
10      4.388346e-02 ||4.174249e+00 -2.086205e-04 ||9.549919e-01 -1.892004e-05 
11      4.171999e-02 ||4.174071e+00 -1.782727e-04 ||9.549814e-01 -1.052970e-05 
12      4.150482e-02 ||4.173914e+00 -1.573058e-04 ||9.549760e-01 -5.370674e-06 
13     